# CAUSALINFERENCE LIBRARY

In [26]:
import numpy as np
import pandas as pd
from causalinference import CausalModel
from causalinference.utils import random_data 

In [27]:
# --- PASO 1: Inicialización del Modelo ---
Y, D, X = random_data(N=1000, K=3) 
cm = CausalModel(Y, D, X)

print("✅ PASO 1: Modelo Causal Inicializado.")
print("\n--- Estadísticas Descriptivas Iniciales (Antes del Ajuste) ---")
print(cm.summary_stats)

✅ PASO 1: Modelo Causal Inicializado.

--- Estadísticas Descriptivas Iniciales (Antes del Ajuste) ---

Summary Statistics

                       Controls (N_c=515)         Treated (N_t=485)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -1.010        1.694        5.008        2.924        6.019

                       Controls (N_c=515)         Treated (N_t=485)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.324        0.892        0.370        0.992        0.737
             X1       -0.269        0.914        0.325        0.898        0.655
             X2       -0.374        0.901        0.293        0.937        0.726



In [28]:
# --- PASO 2: Estimación del Propensity Score (CORREGIDO) ---

cm.est_propensity(lin='all')
print("\n✅ PASO 2: Propensity Score estimado.")

# ✅ Mostrar resultados reales del modelo logístico
print("\n--- Resultados del Modelo de Propensity Score ---")
print(cm.propensity)

print("\n--- Estadísticas descriptivas después del ajuste ---")
print(cm.summary_stats)


✅ PASO 2: Propensity Score estimado.

--- Resultados del Modelo de Propensity Score ---

Estimated Parameters of Propensity Score

                    Coef.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
     Intercept     -0.062      0.079     -0.780      0.435     -0.217      0.094
            X0      1.122      0.096     11.725      0.000      0.935      1.310
            X1      1.035      0.098     10.593      0.000      0.844      1.227
            X2      1.108      0.099     11.227      0.000      0.915      1.301


--- Estadísticas descriptivas después del ajuste ---

Summary Statistics

                       Controls (N_c=515)         Treated (N_t=485)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -1.010        1.694        5.008        2

In [40]:
# --- PASO 3: Trimming y Validación ---

cm.trim() 
print("\n✅ PASO 3: Trimming aplicado (Muestra ajustada para solapamiento).")
print(cm.summary_stats)


✅ PASO 3: Trimming aplicado (Muestra ajustada para solapamiento).

Summary Statistics

                       Controls (N_c=378)         Treated (N_t=396)             
       Variable         Mean         S.d.         Mean         S.d.     Raw-diff
--------------------------------------------------------------------------------
              Y       -0.476        1.366        3.967        2.199        4.443

                       Controls (N_c=378)         Treated (N_t=396)             
       Variable         Mean         S.d.         Mean         S.d.     Nor-diff
--------------------------------------------------------------------------------
             X0       -0.276        0.814        0.147        0.861        0.504
             X1       -0.133        0.862        0.198        0.866        0.383
             X2       -0.150        0.916        0.160        0.957        0.331



In [38]:
# --- PASO 4: Estimación de Efectos Causaless ---

print("\n✅ PASO 4: Ejecutando Estimadores Causaless...")
cm.est_via_weighting()
cm.est_via_matching(matches=1)
cm.est_via_ols(adj=2)
print(cm.estimates)


✅ PASO 4: Ejecutando Estimadores Causaless...

Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.902      0.090     32.275      0.000      2.725      3.078

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      3.158      0.156     20.236      0.000      2.852      3.464
           ATC      2.675      0.193     13.847      0.000      2.297      3.054
           ATT      3.620      0.175     20.697      0.000      3.277      3.962

Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      2.897     

In [32]:
# --- PASO 5: Análisis de Resultados ---

print("\n✅ PASO 5: Resultados Finales de Estimación Causal (ATE, ATT, ATC) ---")
print(cm.estimates)


✅ PASO 5: Resultados Finales de Estimación Causal (ATE, ATT, ATC) ---

Treatment Effect Estimates: Weighting

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      3.068      0.095     32.270      0.000      2.881      3.254

Treatment Effect Estimates: Matching

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
           ATE      3.337      0.150     22.294      0.000      3.043      3.630
           ATC      2.878      0.177     16.225      0.000      2.530      3.226
           ATT      3.782      0.172     21.940      0.000      3.444      4.120

Treatment Effect Estimates: OLS

                     Est.       S.e.          z      P>|z|      [95% Conf. int.]
--------------------------------------------------------------------------------
      

<div style="background-color:#f0f0f0; padding:16px; border-radius:8px;">
<span style="font-size:22px; font-weight:bold;">Resultados Finales de Estimación Causal (ATE, ATT, ATC)
</div>

<div style="background-color:#f0f0f0; padding:16px; border-radius:8px;">
<span style="font-size:22px; font-weight:bold;">Libreria Causalinference - Python
</div>

## Estimadores Causales Fundamentales

A continuación se muestran las formulaciones matemáticas de los principales métodos de estimación causal implementados en `CausalModel`.

---

### 1. Weighting (IPW) — *Inverse Probability Weighting*

El estimador IPW (ponderación por probabilidad inversa) utiliza los *propensity scores* para reponderar las observaciones y estimar el ATE:


$$
\widehat{\tau}_{IPW} 
= 
\frac{1}{n}
\sum_{i=1}^{n}
\left(
\frac{D_i Y_i}{\widehat{e}(X_i)}
-
\frac{(1 - D_i) Y_i}{1 - \widehat{e}(X_i)}
\right)
$$

****
$\widehat{\tau}{IPW} = \frac{1}{n} \sum{i=1}^{n} \left( \frac{D_i Y_i}{\widehat{e}(X_i)} - \frac{(1 - D_i) Y_i}{1 - \widehat{e}(X_i)} \right)$






donde:

- $D_i \in \{0,1\}$ indica si la unidad $i$ recibió el tratamiento,  
- $Y_i$ es el resultado observado,  
- $\widehat{e}(X_i) = \Pr(D_i = 1 \mid X_i)$ es el *propensity score* estimado.

 **Interpretación:** el IPW repondera cada individuo por la probabilidad inversa de recibir el tratamiento que efectivamente recibió, equilibrando las covariables entre grupos tratados y no tratados.

---

### 2. Matching — *Nearest Neighbor Matching*

El estimador por emparejamiento (*matching*) compara unidades tratadas y no tratadas con valores similares de covariables o *propensity score*:

$$
\widehat{\tau}_{Matching}
= 
\frac{1}{N_T} 
\sum_{i: D_i = 1}
\left(
Y_i - \frac{1}{|J(i)|}
\sum_{j \in J(i)} Y_j
\right)
$$

donde:

- $J(i)$ es el conjunto de vecinos más cercanos al individuo $i$ en el grupo de control,  
- $|J(i)|$ es el número de vecinos usados (por ejemplo, 1 en el caso *1-NN*),  
- $N_T$ es el número total de unidades tratadas.

 **Interpretación:** el efecto del tratamiento se obtiene promediando las diferencias de resultados entre cada tratado y sus controles emparejados.

---

### 3. Regresión (OLS) — *Outcome Regression Estimator*

El estimador por regresión lineal ajusta un modelo para el resultado en función del tratamiento y las covariables:

$$
Y_i = \beta_0 + \tau D_i + \beta^\top X_i + \varepsilon_i
$$

El efecto causal promedio estimado (ATE) es el coeficiente $\widehat{\tau}$ asociado a la variable de tratamiento $D_i$:

$$
\widehat{\tau}_{OLS} = \widehat{\beta}_D
$$

 **Interpretación:** este enfoque asume que, condicionalmente a $X$, la relación entre $D$ y $Y$ es lineal, y que no hay variables omitidas que confundan el efecto.

---

### 4. Blocking — *Stratification (Subclassification) Estimator*

El método de *blocking* o estratificación divide el espacio de los *propensity scores* en $K$ bloques homogéneos:

$$
\widehat{\tau}_{Blocking}
=
\sum_{k=1}^{K}
\frac{n_k}{n}
\left(
\overline{Y}_{1k} - \overline{Y}_{0k}
\right)
$$

donde:

- $n_k$ es el número de observaciones en el bloque $k$,  
- $\overline{Y}_{1k}$ y $\overline{Y}_{0k}$ son las medias de $Y$ para tratados y controles dentro del bloque $k$.

💡 **Interpretación:** el efecto se estima dentro de cada estrato y luego se promedia ponderando por el tamaño de cada bloque, garantizando comparabilidad local.

---

 **Resumen conceptual:**

| Método | Idea central | Dependencia de modelo | Supuestos clave |
|:--------|:-------------|:----------------------|:----------------|
| Weighting (IPW) | Reponderar unidades por $1/e(X)$ | Medio | Correcta especificación de $e(X)$ |
| Matching | Comparar unidades similares | Bajo | Buen emparejamiento / soporte común |
| OLS | Modelar $Y$ condicional a $(D, X)$ | Alto | Linealidad y homocedasticidad |
| Blocking | Estratificar por $e(X)$ | Bajo | Balance dentro de estratos |

---
